In [3]:
!pip install sentence-transformers chromadb gradio transformers accelerate arxiv


In [4]:
import gradio as gr
import arxiv
import torch
from sentence_transformers import SentenceTransformer
import chromadb
from transformers import pipeline


In [5]:
!pip install arxiv==2.1.0


In [6]:
chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name="research_papers")
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# using Hugging Face LLM
hf_model = pipeline(
    "text-generation",
    model="HuggingFaceH4/zephyr-7b-beta",
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

print("Models and vector store initialized!")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Device set to use cpu


Models and vector store initialized!


In [7]:
def fetch_papers_from_arxiv(topic, max_results=5):
    """
    Automatically fetch recent papers from arXiv based on the topic.
    """
    search = arxiv.Search(
        query=topic,
        max_results=max_results,
        sort_by=arxiv.SortCriterion.Relevance
    )

    papers = []
    for result in search.results():
        papers.append({
            "title": result.title,
            "summary": result.summary[:500]
        })
    return papers

# Example Test
papers = fetch_papers_from_arxiv("transformer architecture")
for p in papers[:2]:
    print(f"🧾 {p['title']}\n➡️ {p['summary'][:150]}...\n")



/tmp/ipython-input-1172752914.py:12: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for result in search.results():


🧾 TurboViT: Generating Fast Vision Transformers via Generative Architecture Search
➡️ Vision transformers have shown unprecedented levels of performance in
tackling various visual perception tasks in recent years. However, the
architect...

🧾 Differentiable Neural Architecture Transformation for Reproducible Architecture Improvement
➡️ Recently, Neural Architecture Search (NAS) methods are introduced and show
impressive performance on many benchmarks. Among those NAS studies, Neural
...



In [8]:
def store_papers(papers):
    for i, p in enumerate(papers):
        collection.add(
            ids=[str(len(collection.get()['ids']) + i)],
            documents=[p["summary"]],
            metadatas=[{"title": p["title"]}]
        )
    print(f"✅ Stored {len(papers)} papers in ChromaDB.")

# Example: Fetch and store
store_papers(fetch_papers_from_arxiv("artificial intelligence"))


/tmp/ipython-input-1172752914.py:12: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for result in search.results():
/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:00<00:00, 102MiB/s]


✅ Stored 5 papers in ChromaDB.


In [9]:
def research_chat(message, history):
    try:
        context = " ".join([m[0] + " " + m[1] for m in history[-3:]]) + " " + message

        #Query top relevant papers
        results = collection.query(query_texts=[context], n_results=3)
        top_docs = results['documents'][0]
        top_titles = [m['title'] for m in results['metadatas'][0]]

        # Build base summary
        base_response = f"📚 Based on your query, I found {len(top_docs)} related papers:\n"
        for i, (t, d) in enumerate(zip(top_titles, top_docs)):
            base_response += f"\n{i+1}. **{t}** — {d[:150]}...\n"

        # Generate human-like answer
        prompt = (
            "You are a research assistant summarizing papers simply and helpfully.\n"
            f"User context: {context}\n\n"
            f"Relevant research:\n{base_response}\n\nAnswer clearly:"
        )

        output = hf_model(prompt, max_new_tokens=200, do_sample=True)[0]['generated_text']
        answer = output.split("Answer clearly:")[-1].strip()
        return answer

    except Exception as e:
        return f"⚠️ Error: {str(e)}"


In [11]:
with gr.Blocks(theme="default") as demo:
    gr.Markdown("## 🧠 Smart AI Research Assistant")

    topic_input = gr.Textbox(label="Enter Research Topic")
    fetch_btn = gr.Button("📡 Fetch & Store Papers")
    fetch_output = gr.Textbox(label="Fetched Paper Titles")

    chatbot = gr.Chatbot(label="Chat about your research topic")
    msg = gr.Textbox(label="Ask your research question...")
    clear = gr.ClearButton([msg, chatbot])

    state = gr.State([])

    def fetch_and_store(topic):
        papers = fetch_papers_from_arxiv(topic)
        store_papers(papers)
        titles = "\n".join([p["title"] for p in papers])
        return f" Added papers:\n{titles}"

    fetch_btn.click(fetch_and_store, inputs=topic_input, outputs=fetch_output)
    msg.submit(research_chat, inputs=[msg, state], outputs=chatbot, preprocess=False)

demo.launch(share=True)


/tmp/ipython-input-3891859948.py:8: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Chat about your research topic")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://01e7e8c3cd80023b21.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
